# Trabajo práctico 8 - Segmentación

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

A partir de la imagen de bloques encontrar, mediante alguno de los métodos mencionados o combinación de ellos (inclusive pueden utilizar operaciones morfológicas como las vistas anteriormente) la mejor segmentación de los bloques respecto del resto de las piezas.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show() 

In [ ]:
#Cargamos la imagen a analizar
IMG_NAME = 'Piezas2.png'
img = cv.imread(IMG_NAME)
imgGray = cv.imread(IMG_NAME, 0)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
imgWidth = img.shape[1]
imgHeight = img.shape[0]

print("Img Width: {} \t Img Height: {}".format(imgWidth, imgHeight))
plotter(img, 'Original Image')
plotter(imgGray, 'Original Image Grayscale', grayScale=True)

Recortamos la parte de interés, la cual es la mesa con los bloques, lo demás descartamos.

In [ ]:
img = img[100:2200, 350:2500, :]
imgGray = imgGray[100:2200, 350:2500]
plotter(img, 'Original Image')
plotter(imgGray, 'Original Image Grayscale', grayScale=True)

Primero, binarizamos la imagen con Otsu. Para lograr esto utilizamos las siguientes funciones auxiliares

In [ ]:
def plotHistogram(img, title = '', bins = 50):
    plt.figure(figsize=(10,10))
    plt.title('Histogram' + title, fontsize=18, fontweight='bold')
    plt.grid()
    plt.hist(img.ravel(),bins,[0,256], color='orange')

def otsuBinarization(img, thresh = 127):
    ret, imgBin = cv.threshold(img,thresh,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    plotter(imgBin, grayScale=True)
    return imgBin

In [ ]:
plotHistogram(imgGray)

Del histograma podemos observar que un valor de umbral de 125 funcionará bien. Procedemos a binarizar la imagen

In [ ]:
imgBin = otsuBinarization(imgGray, thresh = 125)

Realizamos una apertura para eliminar el ruido en la imagen.

In [ ]:
kernel = np.ones((3,3), np.uint8)
imgOpen = cv.morphologyEx(imgBin.copy(), cv.MORPH_OPEN, kernel, iterations=3)
plotter(imgOpen, 'Open', grayScale=True)

## Watersheed

Para utilizar este algoritmo necesitamos definir tres elementos (marcadores) para no generar sobremarcación, estos elementos son:
* Zona de Fondo Seguro
* Zona de Objetos Seguro
* Zona de Incertezas (Resta de las dos anteriores)

### Fondo Seguro

Para ontener la zona de fondo seguro dilatamos la imagen con un kernel de 6x6. Lo que será el fondo seguro es toda la parte que quede en negro de la imagen.

In [ ]:
kernelDilate = np.ones((6,6), np.uint8)
secureBG = cv.dilate(imgOpen.copy(), kernelDilate, iterations = 3)
plotter(secureBG, 'Secure Background', grayScale=True)

### Objetos Seguros

Primero cerramos la imagen para que no haya puntos negros dentro de los bloques

In [ ]:
kernelClosing = np.ones((10,10), np.uint8)
closing = cv.morphologyEx(imgOpen.copy(), cv.MORPH_CLOSE, kernelClosing, iterations=3)
plotter(closing, 'Closing', grayScale=True)

Aplicamos una transformación de distancia, esta transformada calcula la distancia desde un pixel blanco hasta el pixel oscuro mas cercano. Luego de aplicar esta transformada nos quedamos con los pixeles que tengan una distancia mayor o igual al 70% de la máxima distancia.

In [ ]:
distanceTransform = cv.distanceTransform(closing, cv.DIST_L2, 5)
ret, secureObjects = cv.threshold(distanceTransform, 0.7 * distanceTransform.max(), 255, 0)
plotter(secureObjects, 'Secure Objects', grayScale=True)

### Incertezas

In [ ]:
secureObjects = np.uint8(secureObjects)
unknowns = cv.subtract(secureBG,secureObjects)
plotter(unknowns, 'Unknowns', grayScale=True)

### Etiquetado

In [ ]:
ret, markers = cv.connectedComponents(secureObjects)
markers += 1
markers[unknowns == 255] = 0

plotter(markers, 'Markers')

Ahora con todo esto realizado, aplicamos el algoritmo de Watersheed

In [ ]:
outputWatersheed = img.copy()
markers = cv.watershed(outputWatersheed, markers)
outputWatersheed[markers==1] = [0, 0, 0]
plotter(markers, 'Markers')
plotter(outputWatersheed, 'Result')

## K-Means

Volvemos a cargar las imágenes

In [ ]:
img = cv.imread(IMG_NAME)
imgGray = cv.imread(IMG_NAME, 0)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = img[100:2200, 350:2500, :]
imgGray = imgGray[100:2200, 350:2500]
plotter(img, 'Original Image')
plotter(imgGray, 'Original Image Grayscale', grayScale=True)

Pasamos a la imagen a un array de pixeles en 2D

In [ ]:
pixelValues = img.reshape((-1, 3))
pixelValues = np.float32(pixelValues)

Ahora definimos el criterio de __stopping__

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.2)

Definimos el número de clusters, se probaron varios valores y debido a la performance el mejor que se encontró fue K = 10

In [ ]:
k = 10
_, labels, (centers) = cv.kmeans(pixelValues, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

centers = np.uint8(centers)
labels = labels.flatten()

Convertimos todos los pixeles al color de su centroide

In [ ]:
segmentedImg = centers[labels.flatten()]
segmentedImg = segmentedImg.reshape(img.shape)
plotter(segmentedImg)

A continuación vamos a imprimir la imagen con el valor de K asociado, de esta forma pretendemos encontrar para qué valor de K obtenemos los mejores resultados.

In [ ]:
output = img.copy()
output = output.reshape((-1, 3))
for cluster in range(1,11):
    outputCopy = output.copy()
    outputCopy[labels != cluster] = [0, 0, 0]
    outputCopy = outputCopy.reshape(img.shape)
    plotter(outputCopy, 'K = ' + str(cluster))

Vemos que el valor de K = 3 es el mejor para la situación. Volvemos a imprimir esta imagen y la que se obtuvo por Watersheed para ver los diferentes resultados

In [ ]:
cluster = 3
outputKMeans = img.copy()
outputKMeans = outputKMeans.reshape((-1, 3))
outputKMeans[labels != cluster] = [0, 0, 0]
outputKMeans = outputKMeans.reshape(img.shape)
plotter(outputKMeans, 'Resut K-Means')
plotter(outputWatersheed, 'Result Watersheed')

## Conclusiones

**Watersheed:**

Este algoritmo fue muy performante y fácil de implementar, solo necesitamos definir tres zonas y con eso ya alcanza para obtener un buen resultado. Como podemos ver más arriba la imagen se ve bastante bien, sin ruido y con una pequeño cilindro gris de una de las piezas que se encuentran al lado de los bloques, pero todo lo demas se pudo eliminar de manera correcta, incluso el bloque que se encuentra tapado por otros. Además pudimos deshacernos del borde de madera de la mesa.

**K-Means:**

Este algoritmo no resultó tan performante como el anterior, su resultado contiene mucho ruido pero logró eliminar correctamente las otras piezas y solo dejar marcado los bloques. Con este algoritmo no pudimos eliminar el borde de la mesa, y se ven ciertos pixeles encendidos que corresponden a las otras piezas que no son bloques

Por lo tanto el mejor algoritmo para esta situación es Watersheed, produce un resultado mucho mejor que K-means e incluso con una performance mayor.